In [1]:
import fiftyone as fo


In [2]:
session = fo.launch_app(address="0.0.0.0", port=7778)


导入数据集

In [5]:
name = "pboxGufeng3w"

dataset_dir = "/home/pbox/models/fiftyone"

dataset_type = fo.types.ImageDirectory

dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=dataset_type,
    name=name,
)

 100% |█████████████| 32087/32087 [2.2s elapsed, 0s remaining, 14.7K samples/s]      


In [6]:
dataset.persistent = True 

In [3]:
fo.list_datasets()

['pboxGufeng3w']

In [4]:
name = "pboxGufeng3w"
dataset = fo.load_dataset(name)
# dataset.delete()

clip抽向量，存向量数据库

In [5]:
import fiftyone.brain as fob # ML routines
from fiftyone import ViewField as F # for defining custom views

In [6]:
view = dataset.load_saved_view("finetunetest")

qdrant_index = fob.compute_similarity(
    view,
    brain_key="qpbgf_view",
    backend="qdrant",
    model="zero-shot-classification-transformer-torch",
    embeddings="embeddingx",
    collection_name="fiftyone-pboxgufeng1k",
    name_or_path="OFA-Sys/chinese-clip-vit-base-patch16",
)

/home/pbox/miniconda3/envs/fiftyone/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Computing embeddings...
 100% |███████████████| 1291/1291 [8.7m elapsed, 0s remaining, 2.7 samples/s]      


读取美学评分

In [ ]:
import re
import os


dataset.add_sample_field("aes", fo.FloatField)
dataset_dir = '/cfs_models0/models/fiftyone/'
# 获取文件名的字符串
file_names = os.listdir(dataset_dir)

# 循环遍历文件名列表
for filename in file_names:
    try:
        match = re.search(r'aes_(\d+\.\d+)', filename)
        if match:
            aes_score = match.group(1)
            print(f"AES Score: {aes_score}")
            aes = float(aes_score)
        
            sample = dataset[os.path.join(dataset_dir, filename)]
            sample["aes"]=aes
            print(aes)
            sample.save()
        
    except (IndexError, ValueError):  # 捕获可能的异常
        print(f"文件 {filename} 的名字不符合规则，已跳过.")
        continue  # 继续处理下一个文件名
    except KeyError as e:  
        # 处理 KeyError 异常，打印警告并跳过  
        print(f"KeyError: 跳过")  
        continue 

wd14打标

llama3.1打标

In [ ]:
!python /home/pbox/snake/joy-caption-pre-alpha/app.py /home/pbox/snake/gf

读取tags

In [ ]:
import tqdm
from pathlib import Path
image_types = ('*.jpg', '*.png', '*.jpeg')
image_files = []

path = Path(dataset_dir)
print(path)
for image_type in image_types:
    files = list(path.glob(image_type))
    for file in files:
    # 检查文件名是否以 . 开头
        if not file.name.startswith("."):
            image_files.append(str(file.resolve()))



for image_file in image_files:
    sample = dataset[image_file]
    image_path = Path(image_file)
    tag_file = image_path.with_suffix('.txt')   # 获取tag文件的名字
    if os.path.isfile(tag_file):   # 检查tag文件是否已经存在
        print(f"Tag file '{tag_file}'  exists. Skipping image file: {image_file}")
        with open(tag_file, 'r') as f:
        # 使用逗号分割每个标签
            tags = f.read().split(',')
            # 添加标签到样本的tags属性中
        for tag in tags:
            sample.tags.append(tag.strip())
        sample.save()  # 保存样本的修改
    



    

降维

In [9]:
fob.compute_visualization(dataset,brain_key='vis33',embeddings="embedding",method="umap")

Computing embeddings...
 100% |█████████████| 32087/32087 [8.4m elapsed, 0s remaining, 39.6 samples/s]       
Generating visualization...
UMAP( verbose=True)
Thu Aug 22 15:57:55 2024 Construct fuzzy simplicial set
Thu Aug 22 15:57:56 2024 Finding Nearest Neighbors
Thu Aug 22 15:57:56 2024 Building RP forest with 14 trees
Thu Aug 22 15:57:58 2024 NN descent for 15 iterations
	 1  /  15
	 2  /  15
	 3  /  15
	 4  /  15
	 5  /  15
	Stopping threshold met -- exiting after 5 iterations
Thu Aug 22 15:58:03 2024 Finished Nearest Neighbor Search
Thu Aug 22 15:58:05 2024 Construct embedding


Epochs completed:   0%|            0/200 [00:00]

	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Thu Aug 22 15:58:16 2024 Finished embedding


In [23]:
!fiftyone plugins download https://github.com/jacobmarks/clustering-plugin

  163.0Kb [83.4ms elapsed, ? remaining, 1.9Mb/s]    
Copying plugin '@jacobmarks/clustering' to '/home/pbox/fiftyone/__plugins__/@jacobmarks/clustering'


算奇异值

In [10]:
fob.compute_uniqueness(dataset, embeddings="embedding") 

Computing uniqueness...
Computing neighbors for 32087 embeddings; this may take awhile...
Uniqueness computation complete


In [ ]:
most_unique_view = dataset.sort_by("uniqueness", reverse=True)
session.view = most_unique_view.view() # Most unique images

导出筛选数据集

In [4]:
export_dir = "/home/pbox/models/fiftyoneout"
view = dataset.load_saved_view("finetunetest")
view.export(
    export_dir=export_dir,
    dataset_type=fo.types.FiftyOneDataset,
)

Exporting samples...
 100% |██████████████████| 1291/1291 [4.6s elapsed, 0s remaining, 250.3 docs/s]      


转回tags标签

In [2]:
import json
from pathlib import Path
# JSON文件路径
json_file = Path('/home/pbox/models/fiftyoneout/samples.json')
# 从JSON文件路径中获取基础路径
base_path = json_file.parent

# 加载 JSON 数据
with json_file.open() as f:
    data = json.load(f)
# 遍历每一个 sample
for sample in data['samples']:
    # 提取信息
    tags = sample['tags']
    # caption = sample['caption']
    # 获取文件名
    file_path = base_path / Path(sample['filepath'])
    # 输出文件名
    tag_file = file_path.with_suffix('.txt')
    # 检查文件是否已存在
    # if tag_file.is_file():
    #     print(f"Tag file '{tag_file}' already exists. Skipping...")
    #     continue
    # 将标签和标题保存到同名的 .txt 文件中
    with tag_file.open('w') as f:
        # 写入标题
        # f.write(f"Caption: {caption}\n")
            # 在最后添加 "Chinese ancient" 标签
        tags.append("Chinese ancient")

        # 写入标签
        f.write(f"{', '.join(tags)}\n")
 
